In [1]:
from tatc.analysis import (
    collect_observations,
    collect_downlinks,
    compute_latencies,
    reduce_latencies,
    grid_latencies,
    compute_latencies_optimized
)
from tatc.schemas import (
    Point,
    GroundStation,
    WalkerConstellation,
    TwoLineElements,
    Instrument,
)
from tatc.generation import generate_cubed_sphere_points, generate_cubed_sphere_cells

from datetime import datetime, timedelta, timezone
import pandas as pd

In [2]:

instrument = Instrument(name="Test", field_of_regard=180.0)
orbit = TwoLineElements(
    tle=[
        "1 25544U 98067A   22171.11255782  .00008307  00000+0  15444-3 0  9992",
        "2 25544  51.6448 322.0970 0003980 282.3738 231.6559 15.49798078345636",
    ]
)
satellite = WalkerConstellation(
    name="Test",
    orbit=orbit,
    instruments=[instrument],
    number_satellites=500,
    number_planes=10,
)
station = GroundStation(
    name="Station 1", latitude=0, longitude=180, min_elevation_angle=10
)
start = datetime(2022, 6, 1, tzinfo=timezone.utc)
end = datetime(2022, 6, 2, tzinfo=timezone.utc)

tatc_results_downlinks = pd.concat(
    [
        collect_downlinks([station], sat, start, end)
        for sat in satellite.generate_members()
    ],
    ignore_index=True,
)

tatc_results_observations = pd.concat(
    [
        collect_observations(point, sat, instrument, start, end)
        for point in generate_cubed_sphere_points(5000e3).apply(
            lambda r: Point(
                id=r.point_id,
                latitude=r.geometry.y,
                longitude=r.geometry.x,
            ),
            axis=1,
        )
        for sat in satellite.generate_members()
    ],
    ignore_index=True,
)

0:00:00.089644


In [ ]:
start=datetime.now()
suboptim = compute_latencies(tatc_results_observations, tatc_results_downlinks)
print(datetime.now()-start)

In [ ]:
start=datetime.now()
optim = compute_latencies_optimized(tatc_results_observations, tatc_results_downlinks)
print(datetime.now()-start)

In [ ]:
pd.concat([suboptim,optim]).drop_duplicates(keep=False)